无论是解析数据还是产生输出，几乎每一个有实用价值的程序都会涉及某种形式的文本处理。
## 本章的重点放在有关文本操作的常见问题上，例如拆分字符串、搜索、替换、词法分析以及解析。  
许多任务都可以通过内建的字符串方法轻松解决。但是，更复杂的操作可能会需要用到**正则表达式或者创建完整的解析器**才能得到解决。  
以上所有主题本章都有涵盖。此外，本章还提到了一些同Unicode打交道时用到的技巧。

## 2.1 针对任意多的分隔符拆分字符串

#### 2.1.1问题 我们需要将字符串拆分为不同的字段，但是分隔符（以及分隔符之间的空格)在整个字符串中并不一致。
2.1.2 解决方案字符串对象的**split方法只能处理非常简单的情况**，而且不支持多个分隔符 ，  
对分隔符周围可能存在的空格也无能为力。当需要一些更为灵活的功能时，应该使用re.split()方法  
**注：正则表达式**

In [2]:
line = 'asdf fidk;afed,fjek,asdf,            foo'
import re 
re.split(r'[;,\s]\s*',line)  #方法按照能够匹配的子串将字符串分割后返回列表
# \s匹配任意空白字符，[]用来表示一组字符,单独列出，如 [amk] 匹配 'a'，'m'或'k'

['asdf', 'fidk', 'afed', 'fjek', 'asdf', 'foo']

re.split是很有用的，因为可以为分隔符指定多个模式。例如，在上面的解决方案中，  
分隔符可以是逗号、分号或者是空格符（后面可跟着任意数量的额外空格）。  
只要找到了对应的模式，无论匹配点的两端是什么字段，整个匹配的结果就成为那个分隔符。
最终得到的结果是字段列表，同str.split得到的结果一样。

当使用re.split时，需要小心正则表达式模式中的捕获组(capture group)是否包含在了括号中。  
如果用到了捕获组，那么匹配的文本也会包含在最终结果中。如：

In [3]:
fields = re.split(r'(;|,|\s)\s*', line) 
fields

['asdf', ' ', 'fidk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

在特定的上下文中获取到分隔字符也可能是有用的。例如，也许稍后要用到分隔字符来改进字符串的输出：(分别得到值与分隔符)

In [7]:
values = fields[::2] # 注: [m::n]写法， [m]开始，每跳|n|个取一个, -n逆序取数

In [8]:
delimiters = fields[1::2] + ['']

In [9]:
# 用同样的分隔符重新连接
vd = ''.join(v+d for v, d in zip(values, delimiters))

如果不想在结果中看到分隔字符，但仍然想用括号来对正则表达式模式进行分组，请确保用的是非捕获组  
以**(?:…)**的形式指定。  [python正则中?的含义](https://www.ycpai.cn/python/D8FLchPb.html)
```python
re.split(r'(?:,|;|\s)\s*',line)  #['asdf','fjdk','afed','fjek','asdf','foo']
```

## 2.2 在字符串的开头或结尾处做文本匹配
#### 2.2.1问题我们需要在字符串的开头或结尾处按照指定的文本模式做检查，例如检查文件的扩展名、URL协议类型等。
2.2.2 解决方案有一种简单的方法可用来检查字符串的开头或结尾，只要使用str.**startswith**()和str.**endswith**()
``` python 
filename = 'spam.txt' 
filename.endswith ('.txt')  #True 

url = 'http://www.python.org' 
url.startswith ('http:')  #True 
```
如果需要同时针对多个选项做检查，只需给startswith和endswith提供包含可能选项的元组即可。

这里有另一个例子： 
```python
from urllib.request import urlopen 
def read_data(name):
    if name.startswith(('http:','https:','ftp:')) 
        return urlopen(name).read()
    else: 
        with open(name)as f: 
            return f.read() 
```
奇怪的是，这是Python中需要把元组当成输入的一个地方。如果我们刚好把选项指定在了列表或集合中，请确保首先用tuple将它们转换成元组。示例如下：
```python
choices ['http:','ftp:' ]
url = 'http://www.python.org '
url.startswith (choices) # Error:startswith first arg must be str or a tuple of str,not list
url.startswith (tuple(choices))

startswith()和endswith()方法提供了一种非常方便的方式来对字符串的前缀和后缀做基本的检查。  
类似的操作也可以用切片来完成，但是那种方案不够优雅。例如：
```python
filename = spam.txt' 
filename ='.txt'
url = 'http://www.python.org' 
url[:5] = 'http:' or url[:6] == 'https:'or url[:4]='ftp:
```
可能我们也比较倾向于使用正则表达式作为替代方案,这也行得通，但是通常对于简单的匹配来说有些过于重量级了。  
使用本节提到的技术会更简单，运行得也更快。
最后但同样重要的是，当startswith()和endswith()方法和其他操作（比如常见的数据整理操作)结合起来时效果也很好。  
例如，下面的语句检查目录中有无出现特定的文件：
`if any ( name.endswith(('.c','.h')) for name in listdir(dirname) ):`

## 2.3 利用Shell通配符做字符串匹配
#### 2.3.1问题 当工作在UNIX Shell下时，我们想使用常见的通配符模式（即，*.py、Dat[0-9]*.csv 等)来对文本做匹配。
2.3.2 解决方案 **fnmatch模块**提供了两个函数一fnmatch和fnmatchcase() 可用来执行这样的匹配。使用起来很简单：

In [13]:
from fnmatch import fnmatch,fnmatchcase 

```python 
fnmatch('foo.txt', '*.txt')     #True
fnmatch('foo.txt','?oo.txt')     #True
fnmatch ('Dat45.csv','Dat [0-9]*')   #True
names =['Dat1.csv','Dat2.csv','config.ini','foo.py'] 
[name for name in names if fnmatch (name,'Dat*.csv')]  #找到names中的所有.csv  ->  [Dat1.csv',Dat2.csv']
```

一般来说，fnmatch()的匹配模式所采用的大小写区分规则和底层文件系统相同（根据操作系统的不同而有所不同)。  
如果这个区别对我们而言很重要，就应该使用**fnmatchcase()。它完全根据我们提供的大小写方式来匹配**  
关于这些函数，一个常被忽略的特性是它们在处理非文件名式的字符串时的潜在用途。
例如，假设有一组街道地址，就像这样：
```python
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST 
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY', ] 
```    
可以像下面这样写列表推导式：
```python
[ addr for addr in addresses if fnmatchcase(addr, '* ST')]  #得到以ST结尾的
[ addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9]*CLARK*')]

fnmatch所完成的匹配操作有点介乎于简单的字符串方法和全功能的正则表达式之间。  
如果只是试着在处理数据时提供一种简单的机制以允许使用通配符，那么通常这都是个合理的解决方案。  
如果实际上是想编写匹配文件名的代码，那应该使用gob模块来完成，请参见


## 2.4 文本模式的匹配和查找
#### 2.4.1 问题我们想要按照特定的文本模式进行匹配或查找。
如果想要匹配的只是简单的文字，那么通常只需要用基本的字符串方法就可以了，  
比如str.find()、str.endswith()、str.startswith()或类似的函数。  
对于更为复杂的匹配则需要使用正则表达式以及模块。  
 为了说明使用**正则表达式的基本流程**，假设我们想匹配以数字形式构成的日期，比如“11/27/2012”。示例如下：

In [2]:
import re

In [5]:
text1 = '11/27/2012'
text2 = 'Now 27,2012'
#Simple matching:\d+ means match one or more digits 
if re.match(r'\d+/\d+/\d+',text1):   #text2不符合该匹配模式
    print('yes')
else:
    print('no')

yes


如果打算针对**同一种模式做多次匹配**，那么通常会先将正则表达式模式预编译成一个**模式对象**。例如：

In [9]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)') #当定义正则表达式时，我们常会将部分模式用括号包起来的方式引入捕获组。
if datepat.match(text1):  
    print('yes t1')
elif (datepat.match(text2)):
    print('yes t2')

yes t1


In [14]:
#捕获组通常能简化后续对匹配文本的处理，因为每个组的内容都可以单独提取出来  ()里的内容。
d = datepat.match(text1)
d.group(1) #输出'11/27/2012'中的11 , (0)是全部内容
month, day, year = d.groups()

`match方法`总是尝试在`字符串的开头`找到匹配项。如果想**针对整个文本搜索出所有的匹配项**，那么就应该使用**findall**方法。例如：

In [8]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepat.findall(text) #注意findall和match的区别

['11/27/2012', '3/13/2013']

findall方法搜索整个文本并找出所有的匹配项然后将它们以列表的形式返回。  
如果想以`迭代的方式找出匹配项`，可以使用`finditer`方法

In [16]:
for d in datepat.finditer(text):
    print(d.groups())

('11', '27', '2012')
('3', '13', '2013')


本节展示了re模块来对文本做匹配和搜索的基础。基本功能是首先用re.compile.对模式进行编译，  
然后使用像match()、findall或finditer()这样的方法做匹配和搜索。
当指定模式时我们通常会使用原始字符串，比如r'(d+)/(d+)/(d+)'。这样的字符串不会对反斜线字符转义，这在正则表达式上下文中会很有用。否则，我们需要用双反斜线来表示一个单独的\'，例如'\\d+'。

## 2.5 查找和替换文本  
#### 2.5.1问题我们想对字符串中的文本做查找和替换。
2.5.2 解决方案对于简单的文本模式，使用`str.replace()`即可。例如：

In [17]:
text ='yeah,but no,but yeah,but no,but yeah' 
text.replace('yeah','yep')

'yep,but no,but yep,but no,but yep'

针对更为复杂的模式，可以使用`re模块中的sub函数`/方法。  
假设我们想把日期格式从“11/27/2012”改写为“2012-11-27”。示例如下：

In [18]:
text = 'Today is 11/27/2012.PyCon starts 3/13/2013.' 
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2',text)
#如果打算用相同的模式执行重复替换，可以考虑先将模式编译以获得更好的性能。
# datepat = re.compile(r'(\d+)/(\d+)/(\d+)') 
# datepat.sub(r'\3-\1-\2',text)

'Today is 2012-11-27.PyCon starts 2013-3-13.'

Sub的第1个参数是要匹配的模式，第2个参数是要替换上的模式  
类似\3”这样的反斜线加数字的符号代表着模式中捕获组的数量。  
对于更加复杂的情况，可以指定一个**替换回调函数**。

替换回调函数的输入参数是一个匹配对象，由match或find返回。  
用.group方法来提取匹配中特定的部分。这个函数应该返回替换后的文本。  
除了得到替换后的文本外，如果还想知道一共完成了多少次替换，可以使用re.subn()。如 ：newtext,n = datepat.subn(r\3-\1-\2',text) # n为2  
最有技巧性的地方在于指定正则表达式模式，需要多加练习

## 2.6 以不区分大小写的方式对文本做查找和替换  
#### 2.6.1 问题我们需要以不区分大小写的方式在文本中进行查找，可能还需要做替换。
2.6.2 解决方案要进行不区分大小写的文本操作，我们需要使用re模块并且对各种操作都要加上**re.IGNORECASE**标记。例如：

In [20]:
text = 'UPPER PYTHON,lower python,Mixed Python'
re.findall('python',text,flags=re.IGNORECASE)  # PYTHON','python','Python'
re.sub('python','PY',text,flags=re.IGNORECASE) #不区分大小写将python替换

'UPPER PY,lower PY,Mixed PY'

上例存在一种局限性，就是待替换的文本与匹配的文本(如python、Python)大小写并不吻合。  
如果需要修正则需要一个支撑函数(support function)

In [1]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isuppper():
            return word.uppper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

In [ ]:
text = 'UPPER PYTHON,lower python,Mixed Python'
re.sub('python',matchcase('snake'),text,flags=re.IGNORECASE) #按照匹配规则的源格式进行替换  (大写对大写。。。)

## 2.7 定义实现最短匹配的正则表达式  
#### 2.7.1问题：正在尝试用正则表达式对文本模式做匹配，但识别出来的是最长的可能匹配。相反，我们想将其修改为找出最短的可能匹配。
2.7.2 解决方案这个问题通常会在匹配的文本被一对开始和结束分隔符包起来的时候出现（例如带引号的字符串)。为了说明这个问题，请看下面的例子：


In [2]:
import re
str_pat = re.compile(r'\"(.*)\"')
text1 =  'Computer says "no."'
str_pat.findall(text1)

['no.']

In [19]:
text2 = 'Computer says "no." Phone says "yes."'
name =  str_pat.findall(text2)

['no." Phone says "yes.']


在这个例子中，模式(.*)尝试去匹配包含在引号中的文本。但是，`*操作符在正则表达式中采用的是贪心策略`，所以`匹配过程是基于找出最长的可能匹配来进行`的。因此，在text2的例子中，它错误地匹配成2个被引号包围的字符串。  
要解决这个问题，只要在模式中的`*操作符后加上？修饰符`就可以了。这么做使得匹配过程不会以贪心方式进行，也就会产生出最短匹配了  
示例如下：

In [24]:
str_pat2 = re.compile(r'\"(.*?)\"')
str_pat2.findall(text2)

['no.', 'yes.']

本节提到了一个当编写含有`句点（.)字符`的正则表达式时常会遇到的问题。  
在模式中， 句点`除了换行符之外可匹配任意字符`。但是，如果以开始和结束文本（比如说引号）   
将句点括起来的话，在匹配过程中将尝试找出最长的可能匹配结果。这会导致匹配时跳过多个开始或结束文本，  
而将它们都包含在最长的匹配中。在*或+后添加一个？， 会强制将匹配算法调整为寻找最短的可能匹配。

## 2.8 编写多行模式的正则表达式
#### 2.8.1 问题我们打算用正则表达式对一段文本块做匹配，但是希望在进行匹配时能够跨越多行。
2.8.2 解决方案这个问题一般出现在希望使用句点（.)来匹配任意字符，但是忘记了句点并不能匹配换行符时。
要解决这个问题，可以添加对换行符的支持。示例如下：  
text2 = '''/* this is a multiline comment *'  
multiline comment */ '''
```python
comment=re.compile(r'/\*((?:.|\n)*?)\*/') 
comment.findall(text2) #[this is a\n multiline comment ' 
```
在这个模式中，(？：.)指定了一个非捕获组（即，这个组只做匹配但不捕获结果，也不会分配组号)。


re.compile函数可接受一个有用的标记 - re.DOTALL。这使得正则表达式中的句点()  
可以匹配所有的字符，也包括换行符。例如：  
comment=re.compile(r'/\*(.*？)\*/'，re.DOTALL)  
对于简单的情况，使用re.DOTALL标记就可以很好地完成工作。但是如果要处理极其复杂的模式，  
或者面对的是如2.l8节中所描述的为了做分词(tokenizing)而将单独的正则表达式合并在一起的情况，  
如果可以选择的话，通常更好的方法是定义自己的正则表达式模式，这样它无需额外的标记也能正确工作。

## 2.9 将Unicode文本统一表示为规范形式  
#### 2.9.1 问题我们正在同Unicode字符串打交道，但需要确保所有的字符串都拥有相同的底层表示。
2.9.2 解决方案在Unicode中，有些特定的字符可以被表示成多种合法的代码点序列。为了说明这个问题，请看下面的示例：

In [27]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
print(s1)
print(s2)
print((s1 ==s2))  #输出相同但字符不同

Spicy Jalapeño
Spicy Jalapeño
False


这里的文本以两种形式呈现。  
第一种使用的是字符 的全组成(fully composed)形式（U+00F1)。  
第二种使用的是拉丁字母“n”紧跟着一个~”组合而成的字符（U+0303)。  
对于**一个比较字符串的程序来说，同一个文本拥有多种不同的表示形式是个大问题。  
为了解决这个问题，应该先将文本统一表示为规范形式**，这可以通过`unicodedata模块`


In [28]:
import unicodedata 
t1 = unicodedata.normalize('NFC',s1)  
t2 = unicodedata.normalize('NFC',s2) 
t1==t2  #true

True

normalize()的第一个参数指定了字符串应该如何完成规范表示。  
NFC表示字符应该是全组成的（即，如果可能的话就使用单个代码点）。NFD表示应该使用组合字符，每个字符应该是能完全分解开的。  
Python还支持NFKC和NFKD的规范表示形式，它们为处理特定类型的字符增加了额外的兼容功能。  

对于任何需要确保以规范和一致性的方式处理Unicode文本的程序来说，  
规范化都是重要的一部分。**尤其是在处理用户输入时接收到的字符串时，  
此时你无法控制字符串的编码形式**，那么规范化文本的表示就显得更为重要了。  
在对文本进行过滤和净化时，规范化同样也占据了重要的部分。例如，假设想从某些  
文本中去除所有的音符标记（可能是为了进行搜索或匹配）：
```PYTHON
t1 = unicodedata.normalize('NFD',s1) 
''.join(c for c in t1 if not unicodedata.combining(c)) #'Spicy Jalapeno'  
```
最后一个例子展示了unicodedata模块的另一个重要功能一用来**检测字符是否属于某个字符类别**。  
使用工具combining函数可对字符做检查，判断它是否为一个组合型字符。这个模块中还有一些函数可用来查找字符类别、检测数字字符等。
## 2.10 用正则表达处理Unicode字符  
#### 2.10.1问题我们正在用正则表达式处理文本，但是需要考虑处理Unicode字符。
2.10.2 解决方案默认情况下re模块已经对某些Unicode字符类型有了基本的认识。例如，\d已经可以匹配任意Unicode数字字符了  
如果需要在模式字符串中包含指定的Unicode字符，可以针对Unicode字符使用转义序列（例如\uFFFF或\UFFFFFFF)。  
比如，这里有一个正则表达式能在多个不同的阿拉伯代码页中匹配所有的字符：  
arabic = re.compile('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+')
  
当执行匹配和搜索操作时，一个好主意是首先将所有的文本都统一表示为标准形式（见2.9节)。  
但是，同样重要的是需要注意一些特殊情况。例如，当不区分大小写的匹配和大写转换（case folding)匹配联合起来时，考虑会出现什么行为...
  
**把Unicode和正则表达式混在一起使用绝对是个能让人头痛欲裂**的办法。  
如果真的要这么做，应该考虑安装第三方的正则表达式库(htp://pypi..python..org/pypi/regex),  
这些第三方库针对Unicode大写转换提供了完整的支持，还包含其他各种有趣的特性，包括近似匹配。

## 2.11 从字符串中去掉不需要的字符  
#### 2.11.1 问题我们想在字符串的开始、结尾或中间去掉不需要的字符，比如说空格符。
2.11.2 解决方案`strip`方法可用来从字符串的开始和结尾处去掉字符。  
`lstrip()`和`rstrip()`可分别从左或从右侧开始执行去除字符的操作。默认情况下这些方法去除的是空格符，但也可以指定其他的字符。例如：

In [31]:
s = ' hello world \n' 
print( s.strip())
print( s.lstrip())
print( s.rstrip())

hello world
hello world 

 hello world


In [32]:
t ='-----he110====='
t.lstrip('-')

'he110====='

当我们读取并整理数据以待稍后的处理时常常会用到这类strip()方法。   
例如，可以用它们来去掉空格、移除引号等。  
需要注意的是，**去除字符的操作并不会对位于字符串中间的任何文本起作用**。例如：'hello     world' #中间空格无法取消  
如果要对里面的空格执行某些操作，应该使用其他技巧，比如使用`replace()方法`或正则表达式替换。例如：

In [34]:
print( s.replace(' ',""))
re.sub('\s+', ' ', s)


helloworld



' hello world '

我们通常会遇到的情况是将去除字符的操作同某些迭代操作结合起来，  
比如说从文件中读取文本行。如果是这样的话，那就到了**生成器表达式**大显身手的时候了。例如
```python
 with open(filename)as f: 
    lines = (line.strip() for line in f) 
    for line in lines: 
```
这里，表达式lines = 的作用是完成数据的转换。它很高效，因为这里并没有先将数据读取到任何形式的临时列表中。  
它只是创建一个迭代器，在所有产生出的文本行上都会执行strip操作。  
对于更高级的strip操作，应该转而使用translate(方法。请参见下一节以获得进一步的细节。

## 2.12文本过滤和清理  
#### 2.12.1问题某些无聊的脚本小子在Wb页面表单中填入了“pth6~~”这样的文本，我们想以某种方式将其清理掉。
2.12.2解决方案文本过滤和清理所涵盖的范围非常广泛，涉及文本解析和数据处理方面的问题。  
在非常简单的层次上，我们可能会用基本的字符串函数（例如str.upper和str.lower将文本转换为标准形式。  
简单的替换操作可通过str.replace()或re.sub来完成，它们把重点放在移除或修改特定的字符序列上。也可以利用unicodedata.normalize()来规范化文本， 如2.9节所示。  
然而我们可能想更进一步。**比方说也许想清除整个范围内的字符，或者去掉音符标志**。  
要完成这些任务，可以使用常被忽视的`str.translate`方法。为了说明其用法，假设有如下这段混乱的字符串：

In [37]:
s = 'python\fis\tawesome\r\n'
# 第一步是清理空格。要做到这步，先建立一个小型的转换表，然后使用translate()方法：
remap ={ 
        ord('\t'):' ',
        ord('\f'):' ',
        ord('\r'):None  #Deleted
}
a = s.translate(remap)
print(a)

python is awesome



可以看到，类似\t和\f这样的空格符已经被重新映射成一个单独的空格。  
回车符\r已经完全被删除掉了。可以利用这种重新映射的思想进一步构建出更加庞大的转换表。例如，我们把所有的Unicode组合字符都去掉：

In [38]:
import unicodedata 
import sys 
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining (chr(c)))
b=unicodedata.normalize('NFD',a)
print(b)
print(b.translate(cmb_chrs))

python is awesome




在这个例子中，我们使用dict.fromkeys()方法构建了一个将每个Unicode组合字符都映射为None的字典。  
原始输入会通过unicodedata.normalize()方法转换为分离形式，然后再通过translate()方法删除所有的重音符号。  
我们也可以利用相似的技术来去掉其他类型的字符（例如控制字符)。  
下面来看另一个例子。这里有一张转换表将所有的Unicode十进制数字字符映射为它们对应的ASCI版本：

In [40]:
digitmap = {c:ord('0') + unicodedata.digit(chr(c)) 
            for c in range(sys.maxunicode) 
            if unicodedata.category(chr(c))=='Nd'  }
len(digitmap)

650

In [41]:
#Arabic digits 
x= '\u0661\u0662\u0663' 
x.translate(digitmap)

'123'

另一种用来清理文本的技术涉及/O解码和编码函数。  
大致思路是首先对文本做初步的清理，然后通过结合encode()和decode()操作来修改或清理文本。示例如下：  
```python
# a 'python is awesome\n'   #这里的h为带^的h
b = unicodedata.normalize('NFD',a) 
b.encode('ascii','ignore').decode('ascii') 
#b 'python is awesome\n' 
```
这里的normalize方法先对原始文本做分解操作。后续的ASCII编码/解码只是简单地一次性丢弃所有不需要的字符。很显然，这种方法只有当我们的最终目标就是ASCⅡ形式的文本时才有用。

文本过滤和清理的一个主要问题就是运行时的性能。一般来说操作越简单，运行得就越快。  
对于**简单的替换操作，用str.replace**通常是最快的方式一即使必须多次调用它也是如此。比方说如果要清理掉空格符  
另一方面，如果需要做任何高级的操作，比如**字符到字符的重映射或删除，那么translate() 方法还是非常快**的。  
从整体来看，我们应该在具体的应用中去进一步揣摩性能方面的问题。不幸的是，想在技术上给出一条“放之四海而皆准”的建议是不可能的，所以应该尝试多种不同的方法，然后做性能统计分析。  
尽管本节的内容主要关注的是文本，但类似的技术也同样适用于字节对象(byte),这包括简单的替换、翻译和正则表达式。

## 2.13 对齐文本字符串  
## 2.13.1问题我们需要以某种对齐方式将文本做格式化处理。
2.13.2 解决方案对于基本的字符串对齐要求，可以使用字符串的`ljust()、rjust和center`()方法。示例如下：

In [54]:
text = 'Hello World' 
text.ljust(20)  #左对齐，右侧填充空格 ，可接受填充字符 （20，'*'）

'Hello World         '

In [58]:
text.center(20,'-')

'----Hello World-----'

`format`函数也可以用来轻松完成对齐的任务。需要做的就是合理利用<、>，或^字符以及一个期望的宽度值如：                                                                                                                            

In [66]:
t = format(text,'>20') #右对齐 ； 
print(t)

         Hello World


In [60]:
format(text,'-<20s') #想包含空格之外的填充字符，可以在对齐字符之前指定 ； < s为格式

'Hello World---------'

当格式化多个值时，这些格式化代码也可以用在format方法中。

In [64]:
'{:->10s}{:->10s}'.format('Hello','World') # -为填充符 ；  ：> s为格式

'-----Hello-----World'

**format()的好处之一是它并不是特定于字符串的**。它能作用于任何值，这使得它更加通用。例如，**可以对数字做格式化处理**：

In [69]:
x = 1.2345 
format(x,'>10')

'    1.2345'

在比较老的代码中，通常会发现%操作符用来格式化文本。例如：  
```python
 '%-20s' % text  #'Hello World 
 '%20s'% text  #'         Hello World'  
 ```
 但是在新的代码中，我们应该会更钟情于使用format()函数或方法。  
 format()比%操作符提供的功能要强大多了。此外，format可作用于任意类型的对象，比字符串的just、rjust以及center()方法要更加通用。

## 2.14 字符串连接及合并  
#### 2.14.1问题我们想将许多小字符串合并成一个大的字符串。
2.14.2解决方案如果想要**合并的字符串在一个序列或可迭代对象**中，那么将它们合并起来的最快方法就是使用`join`方法。示例如下：

In [71]:
parts = ['Is','Chicago','Not','Chicago?']  
' '.join (parts)    # ','.join 用,号连接

'Is Chicago Not Chicago?'

初看上去语法可能显得有些怪异，但是join操作其实是字符串对象的一个方法。  
这么设计的部分原因是因为想要合并在一起的对象可能来自于各种不同的数据序列，  
比如列表、元组、字典、文件、集合或生成器，如果单独在每一种序列对象中实现一个join 方法就显得太冗余了。  
因此只需要指定想要的分隔字符串，然后在字符串对象上使用join方法将文本片段粘合在一起就可以了。  
如果只是想连接一些字符串，一般使用+操作符就足够完成任务了  `'hi' + 'world'`

字符串连接这个主题可能看起来还没有高级到要用一整节的篇幅来讲解，  
但是**程序员常常会在这个问题上做出错误的编程选择，使得他们的代码性能受到影响。  
最重要的一点是要意识到使用`+操作符做大量的字符串连接是非常低效的`**，原因是由于`内存拷贝和垃圾收集产生的影响`。  
特别是你绝不会想写出这样的字符串连接代码： 
```python
S=' ' 
for p in parts: 
    s +=p 
```
这种做法比使用join方法要慢上许多。主要是因为每个+=操作都会创建一个新的字符串对象。  
我们最好先收集所有要连接的部分，最后再一次将它们连接起来。
一个相关的技巧（很漂亮的技巧）是利用生成器表达式（见1.19节）在将数据转换为字符串的同时完成连接操作。示例如下  
```python
data=['ACME',50,91.1]
','.join( str(d) for d in data )
```
对于不必要的字符串连接操作也要引起重视。有时候在技术上并非必需的时候，程序员们也会忘乎所以地使用字符串连接操作。例如在打印的时候：
```python
print(a+':'+b+':'+c) #Ugly 
print(':'.join([a,b,c])) #Still ugly 
print(a,b,c,sep=':')  #    Better!
```

将字符串连接同I/O操作混合起来的时候需要对应用做仔细的分析。例如，考虑如下两段代码：
```python
#Version 1 (string concatenation) 
f.write(chunk1 + chunk2)
#Version 2 (separate I/0 operations) 
f.write(chunk1)
f.write(chunk2)
```
如果这两个字符串都很小，那么第一个版本的代码能带来更好的性能，  
这是因为执行一次/O系统调用的固有开销就很高。另一方面，如果这两个字符串都很大，那么第二个版本的代码会更加高效。  
因为这里**避免了创建大的临时结果，也没有对大块的内存进行拷贝**。  
这里必须再次强调，你需要对自己的数据做分析，以此才能判定哪一种方法可以获得最好的性能。  
最后但也是最重要的是，如果我们编写的`代码要从许多短字符串中构建输出`  
则`应该考虑编写生成器函数，通过yield关键字生成字符串片段`。示例如下：

```python
def sample(): 
    yield 'Is' 
    yield 'Chicago' 
    yield 'Not' 
    yield 'Chicago?'
#关于这种方'法有一个有趣的事实，那就是它不会假设产生的片段要如何组合在一起。
#比如说可以用joi0将它们简单的连接起来：
text =' '.join(sample()) 

#或者，也可以将这些片段重定向到 I/O: 
for part in sample(): 
    f.write(part) 

#又或者我们能以混合的方式将I/O操作智能化地结合在一起：
def combine (source,maxsize):
    parts =[] 
    size = 0 
    for part in source: 
        parts.append(part) 
        size +len(part) 
        if size >maxsize: 
            yield ''.join(parts) 
            parts =[] 
            size =0 
    yield ''.join(parts) 

for part in combine(sample(),100): 
    f.write(part)               #关键在于这里的生成器函数并不需要知道精确的细节，它只是产生片段而已
```

## 2.15 给字符串中的变量名做插值处理  
#### 2.15.1问题我们想创建一个字符串，其中`嵌入的变量名称会以变量的字符串值形式替换掉`。
2.15.2 解决方案Python并不直接支持在字符串中对变量做简单的值替换。但是，这个功能可以通过字符串的format()方法近似模拟出来。示例如下：

In [74]:
mnums =36
s = '{name} has {n} messages.'
s.format (name='Guido',n=mnums)

'Guido has 36 messages.'

另一种方式是，如果要被替换的值确实能在变量中找到，则可以将`format_map`和`vars()`联合起来使用，示例如下：

In [76]:
name = 'GUido' ; n=37
s.format_map(vars())  #

'GUido has 37 messages.'

有关`vars`的一个微妙的特性是它也`能作用于类实例`上

In [86]:
class info:
    def __init__(self, name, n):
        self.name = name
        self.n = n

In [87]:
a = info('Kawaski',1000)
s.format_map(vars(a))

'Kawaski has 1000 messages.'

而format(O和format map()的一个**缺点则是没法优雅地处理缺少某个值的情况**。
```python
s.format (name='Guido') 
#Traceback (most recent call last): File "<stdin>",line 1,in <module> KeyError:'n'
```
避免出现这种情况的一种方法就是单独定义一个带有__missing__方法的字典类，示例如

In [88]:
class safesub(dict): 
    def __missing__ (self,key): 
        return '{' + key + '}'
    
# 现在用这个类来包装传给format map()的输入参数
del n #Make sure n is undefined 
s.format_map(safesub(vars ()))

'GUido has {n} messages.'

如果发现自己在代码中常常需要执行这些步骤，则可以将替换变量的过程隐藏在一个小型的功能函数内，  
这里要采用一种称之为`frame hack`的技巧。示例如：  #注:即需要同函数的栈帧打交道。`sys._getframe`这个特殊的函数可以让我们获得调用函数的栈信息

In [91]:
import sys 
def sub(text):
     return text.format_map(safesub(sys._getframe(1).f_locals))

In [92]:
#现在可以这样编写代码
sname = 'GUIdo' ; sn=37
print(sub('Hello {sname}, {sn} mess, {color} is missing'))

Hello GUIdo, 37 mess, {color} is missing


多年来，由于Python缺乏真正的变量插值功能，由此产生了各种解决方案。作为本节中已给出的解决方案的替代，有时候我们会看到类似下面代码中的字符串格式化操作：  
```python
'%(name) has %(n) messages.' % vars()
```
我们可能还会看到模板字符串(template string)的使用：
```python
import string 
s string.Template('$name has $n messages.')
s.substitute (vars())
```
但是，format()和format_map()方法比上面这些替代方案都要更加现代化，  
我们应该将其作为首选。使用`format`的一个好处是可以`同时得到所有关于字符串格式化方面的功能`（对齐、填充、数值格式化等），而这些功能在字符串模板对象上是不可能做到的。  
在本节的部分内容中还提到了一些有趣的高级特性。  
字典类中鲜为人知的`__missing__方法可用来处理缺少值时的行为`。在safesub类中，我们将该方法定义为将缺失的值以占位符的形式返回，因此这里不会抛出KeyError异常，缺少的那个值会出现在最后生成的字符串中（可能对调试有些帮助)。    

sub函数使用了sys.getframe(I)来返回调用方的栈帧。通过访问属性f_locals来得到局部变量。  
无需赘言，在大部分的代码中都应该避免去和栈帧打交道，但是对于类似完成字符串替换功能的函数来说，这会是有用的。插一句题外话，值得指出的是f_locals是一个字典，它完成对调用函数中局部变量的拷贝。尽管可以修改f_locals的内容，可是修改后并不会产生任何持续性的效果。因此，尽管访问不同的栈帧可能看起来是很邪恶的，但是想意外地覆盖或修改调用方的本地环境也是不可能的。

## 2.16 以固定的列数重新格式化文本  
#### 2.16.1问题我们有一些很长的字符串，想将它们重新格式化，使得它们能按照用户指定的列数来显示。
2.16.2解决方案可以使用`textwrap`模块来`重新格式化文本的输出`。  
例如，假设有如下这段长字符串, 可以用该模块来重新格式化字符串

In [4]:
s = "Look into my eyes,look into my eyes, \
the eyes,the eyes,I the eyes,not around the eyes,\
don't look around the eyes, you're under."

import textwrap  #查看三种不同形式输出
print (s, textwrap.fill(s,70), textwrap.fill(s,40),sep='\n\n')  

Look into my eyes,look into my eyes, the eyes,the eyes,I the eyes,not around the eyes,don't look around the eyes, you're under.

Look into my eyes,look into my eyes, the eyes,the eyes,I the eyes,not
around the eyes,don't look around the eyes, you're under.

Look into my eyes,look into my eyes, the
eyes,the eyes,I the eyes,not around the
eyes,don't look around the eyes, you're
under.


textwrap模块能够以简单直接的方式对文本格式做整理使其适合于打印  
尤其是当希望输出结果能很好地显示在终端上时。关于终端的尺寸大小，可以通过`os.get_terminala_size().columns`来获取  
`fill`方法还有一些额外的选项可以用来控制如何处理制表符、句号等。请参阅API文档

## 2.17 在文本中处理HTML和XML实体  
#### 2.17.1 问题我们想将&entity或&#code这样的HTML或XML实体替换为它们相对应的文本。或者，我们需要生成文本，但是要对特定的字符（比如<，>或&）做转义处理。
2.17.2解决方案如果要生成文本，使用`html.escape`函数来完成替换<or>这样的特殊字符相对来说是比较容易的。例如：

In [7]:
h = 'Elements are written as "<tag>text</tag>".'
import html 
print(h)
print(html.escape(h))
print(html.escape(h, quote=False))

Elements are written as "<tag>text</tag>".
Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.
Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


如果要生成ASCII文本，并且想针对非ASCⅡ字符将它们对应的字符编码实体嵌入到文本中，  
可以在各种同I/O相关的函数`.encode()`中使用errors='xmlcharrefreplace'参数来实现。  
要替换文本中的实体，那就需要不同的方法。如果实际上是在处理HTML或XML,  
首先应该尝试使用一个合适的HTML或XML解析器。一般来说，这些工具在解析的过程中会自动处理相关值的替换，而我们完全无需为此操心。  
如果由于某种原因在得到的文本中带有一些实体，而我们想手工将它们替换掉，通常可以利用各种HTML或XML解析器自带的功能函数和方法来完成  
利用`from html.parser import HTMLParser `等

在生成HTML或XML文档时，适当地对特殊字符做转义处理常常是个容易被忽视的细节。  
尤其是当自己用printO.或其他一些基本的字符串格式化函数来产生这类输出时更是如此。简单的解决方案是使用像html.escape()这样的工具函数。  
如果需要反过来处理文本（即，将HTML或XML实体转换成对应的字符），有许多像xml.sax.saxutils.unescape()这样的工具函数能帮上忙。  
但是，我们需要仔细考察一个合适的解析器应该如何使用。  
例如，如果是处理HTML或XML,像html.parser 或xml.etree.ElementTree这样的解析模块应该已经解决了有关替换文本中实体的细节问题。

## 2.18文本分词
#### 2.18.1问题我们有一个字符串，想从左到右将它解析为标记流（stream of tokens)。
2.18.2 解决方案假设有如下的字符串文本： text = 'foo =23+42\*10' 要对字符串做分词处理，需要做的不仅仅只是匹配模式。  
我们还需要有某种方法来识别出模式的类型。例如，我们可能想将字符串转换为如下的序列对:  

In [9]:
tokens= [('NAME','foo'), ('EQ','='), ('NUM','23'),( 'PLUS','+'),   
    ('NUM','42'),('TIMES','*'),('NUM','10')]

要完成这样的分词处理，第一步是**定义出所有可能的标记，包括空格**。  
这可以通过`正则表达式中的命名捕获组`来实现,在这些正则表达式模式中，形如`?P< TOKENNAME >`这样的约定是用来`将名称分配给该模式`的。这个我们稍后会用到。
示例如下：

In [12]:
import re 
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'

NUM =r'(?P<NUM>\d+)'    # \j将一个普通字符变成特殊字符，例如 \d 表示匹配所有十进制数字 ；匹配前面的子表达式一次或多次，等价于 {1,}
PLUS = r'(?P<PLUS>\+)'    # \ 第二个功能，解除元字符的特殊功能，\+表示匹配其本身
TIMES = r'(?P<TIMES>\*)'

EQ =r'(?P<EQ>=)'
WS =r'(?P<WS>\s+)'   # \s匹配空白字符

master_pat = re.compile('|'.join([NAME,NUM,PLUS,TIMES,EQ,WS]))  #模式

接下来我们使用模式对象的`scanner`方法来完成分词操作。该方法会创建一个扫描对象，  
在给定的文本中重复调用match,一次匹配一个模式。下面这个交互式示例展示了`扫描对象`是如何工作的：

In [13]:
scanner = master_pat.scanner('foo = 42')  
scanner.match()

<re.Match object; span=(0, 3), match='foo'>

In [14]:
_.lastgroup, _.group()

('NAME', 'foo')

In [15]:
scanner.match()

<re.Match object; span=(3, 4), match=' '>

In [16]:
_.lastgroup, _.group()

('WS', ' ')

In [17]:
scanner.match()

<re.Match object; span=(4, 5), match='='>

In [18]:
_.lastgroup, _.group()  #....继续重复该两行代码，可以看到其余效果

('EQ', '=')

要利用这项技术并将其转化为代码，我们可以做些清理工作然后轻松地`将其包含在一个生成器函数中`，示例如下：

In [20]:
from collections import namedtuple 
Token = namedtuple('Token', ['type','value'])

def generate_tokens(pat,text): 
    scanner = pat.scanner(text)   #完成分词操作
    for m in iter(scanner.match, None): 
        yield Token(m.lastgroup, m.group())

#Example use
for tok in generate_tokens(master_pat, 'foo = 23 + 42*10'):
    print(tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='23')
Token(type='WS', value=' ')
Token(type='PLUS', value='+')
Token(type='WS', value=' ')
Token(type='NUM', value='42')
Token(type='TIMES', value='*')
Token(type='NUM', value='10')


如果想以某种方式对标记流做过滤处理，要么定义更多的生成器函数，要么就用生成器表达式。例如，下面的代码告诉我们如何过滤掉所有的空格标记。
```python
tokens = (tok for tok in generate_tokens(master_pat,text) 
                        if tok.type != 'WS') 
```

**对于更加高级的文本解析，第一步往往是分词处理。**要使用上面展示的扫描技术，  
有几个重要的细节需要牢记于心。第一，对于每个可能出现在输入文本中的文本序列，  
都要确保有一个对应的正则表达式模式可以将其识别出来。如果发现有任何不能匹配的文本，扫描过程就会停止。**这就是为什么有必要在上面的示例中指定空格标记(WS)**。  
**这些标记在正则表达式**（即re.compile('join(NAME,NUM,PLUS,TIMES,EQ,WS])) **中的顺序同样也很重要**。当进行匹配时，模块会按照指定的顺序来对模式做匹配。  
因此，**如果碰巧某个模式是另一个较长模式的子串时，就必须确保较长的那个模式要先做匹配**。  
对于更加高级的分词处理，我们应该去看看像PyParsing或PLY这样的包。有关PLY 的例子将在下一节中讲解。

## 2.19 编写一个简单的递归下降解析器
#### 2.19.1 问题我们需要根据一组语法规则来解析文本，以此执行相应的操作或构建一个抽象语法树来表示输入  
语法规则很简单，因此我们倾向于自己编写解析器而不是使用某种解析器框架。
2.19.2 解决方案在这个问题中，我们把重点放在根据特定的语法来解析文本上。  
要做到这些，应该以BNF或EBNF的形式定义出语法的正式规格。比如，对于简单的算术运算表达式，语法看起来是这样的：
```
expr ::= expr + term 
     | expr - term 
     | term 
term ::= term * factor 
     | term / factor 
     | factor 
factor ::= ( expr ) 
     |NUM
```
又或者以EBNF的形式定义如下形式：
```
expr ::= term { (+|-) term}* 
term ::= factor { (*|/factor}* 
factor ::=(expr)
          | INUM
``` 

在EBNF中，部分包括在{...}*中的规则是可选的。*意味着零个或更多重复项（和在正则表达式中的意义相同)。  
现在，如果我们对**BNF**还不熟悉的话，可以把它看做是**规则替换或取代的一种规范形式**，左侧的符号可以被右侧的符号所取代（反之亦然)。  
一般来说，在解析的过程中我们会尝试将输入的文本同语法做匹配，通过BNF来完成各种替换和扩展。  
为了说明， 假设正在解析一个类似于3+4*5这样的表达式。这个表达式首先应该被分解为标记流，这可以使用2.18节中描述的技术来实现。得到的结果可能是下面这样的标记序列：  
`NUM + NUM *NUM` ,  从这里升始，解析过程就涉及通过替换的方式将语法匹配到输入标记上：  ......内容较多，可直接应用意义较小，详细见PDF88页

In [1]:
import pandas as pd  
import time
print(pd.Timestamp(1303608600))

1970-01-01 00:00:01.303608600


In [ ]:
timestamp = 1530600808
time_local = time.localtime(timestamp)
time_local

time.struct_time(tm_year=2018, tm_mon=7, tm_mday=3, tm_hour=14, tm_min=53, tm_sec=28, tm_wday=1, tm_yday=184, tm_isdst=0)

In [15]:
pd.to_datetime([1349720105, 1530600808], unit="s")

DatetimeIndex(['2012-10-08 18:15:05', '2018-07-03 06:53:28'], dtype='datetime64[ns]', freq=None)

## 2.20在字节串上执行文本操作  
#### 2.20.1问题 我们想在**字节**串(Byte String)上执行常见的文本操作（例如，拆分、搜索和替换)。
2.20.2解决方案 字节串已经支持大多数和文本字符串一样的内建操作。例如：

In [5]:
bdata = bytearray(b'Hello World') 
print(bdata[0:5])

print(bdata.startswith(b'Hello')) 
print(bdata.split()) 
print(bdata.replace(b'Hello', b'Hi Python')) 

bytearray(b'Hello')
True
[bytearray(b'Hello'), bytearray(b'World')]
bytearray(b'Hi Python World')


我们可以在字节串上执行正则表达式的模式匹配操作，但是**模式本身需要以字节串的形式指定**

In [2]:
bdata = b'Foo:BAR,SPAM'
import re
re.split(b'[:,]', bdata)  
# 若为re.split('[:,]', bdata)则会报错
#TypeError:can't use a string pattern on a bytes-like object

[b'Foo', b'BAR', b'SPAM']

In [3]:
# 注: 字节字符串的索引操作返回整数而不是单独字符
bdata[0]

70

这种语义上的区别会对于处理面向字节的字符数据有影响  
字节字符串不会提供一个美观的字符串表示，也不能很好的打印出来，除非它们**先被解码**为一个文本字符串  

类似的，也不存在任何适用于字节字符串的格式化操作(如%10d)  
如果你想格式化字节字符串，你得先使用标准的文本字符串，然后将其编码为字节 字符串

In [12]:
print(bdata)
print(bdata.decode('ascii'))

'{:10s} {:10d} {:10.2f}'.format('ACME', 100, 490.1).encode('ascii') #得的的结果为字节字符串

b'Foo:BAR,SPAM'
Foo:BAR,SPAM


b'ACME              100     490.10'